
    Korzystając ze źródła http://archive.ics.uci.edu/ml/datasets.php wybrać jeden system decyzyjny, a następnie dokonać

klasyfikacji metodą KNN dowolnego atrybutu decyzyjnego. Wykorzystując rozwiązanie zadania 1 z regresji KNN znaleźć najlepszą wartość K w następujących podziałach: - 55% system treningowy i 45% system testowy - 60% system treningowy i 40% system testowy - 65% system treningowy i 35% system testowy - 70% system treningowy i 30% system testowy - 75% system treningowy i 25% system testowy - 80% system treningowy i 20% system testowy

Dla każdego podziału zaprezentować poznane wartości miar wydajności modeli - w tym celu należy zaimplementować te funkcjonalności.

In [1]:
import random
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from sklearn.metrics import mean_absolute_error, log_loss, accuracy_score, balanced_accuracy_score
import random
import matplotlib.pyplot as plt

# Wczytanie danych

## Dane są pobrane z http://archive.ics.uci.edu/ml/datasets/Drug+consumption+%28quantified%29

In [25]:
header =['ID' , 'Age' , 'Gender' ,'Education' , 'Country' , 'Ethnicity','Nscore'  ,'Escore','Oscore','Ascore','Cscore','Impulsive', 'SS', 'Alcohol', 'Amphet','Amyl','Benzos','Caff','Cannabis','Choc','Coke','Crack','Ecstasy', 'Heroin' , 'Ketamine' ,'Legalh', 'LSD','Meth','Mushrooms','Nicotine','Semer','VSA'  ]

In [26]:
df = pd.read_csv('drug_consumption.csv', names=header)

In [27]:
df = df.replace('CL0',0).replace('CL1',1).replace('CL2',2).replace('CL3',3).replace('CL5',5).replace('CL4',4).replace('CL6',6)

In [28]:
df

ID      Age   Gender  Education  Country  Ethnicity   Nscore   Escore  \
0        1  0.49788  0.48246   -0.05921  0.96082    0.12600  0.31287 -0.57545   
1        2 -0.07854 -0.48246    1.98437  0.96082   -0.31685 -0.67825  1.93886   
2        3  0.49788 -0.48246   -0.05921  0.96082   -0.31685 -0.46725  0.80523   
3        4 -0.95197  0.48246    1.16365  0.96082   -0.31685 -0.14882 -0.80615   
4        5  0.49788  0.48246    1.98437  0.96082   -0.31685  0.73545 -1.63340   
...    ...      ...      ...        ...      ...        ...      ...      ...   
1880  1884 -0.95197  0.48246   -0.61113 -0.57009   -0.31685 -1.19430  1.74091   
1881  1885 -0.95197 -0.48246   -0.61113 -0.57009   -0.31685 -0.24649  1.74091   
1882  1886 -0.07854  0.48246    0.45468 -0.57009   -0.31685  1.13281 -1.37639   
1883  1887 -0.95197  0.48246   -0.61113 -0.57009   -0.31685  0.91093 -1.92173   
1884  1888 -0.95197 -0.48246   -0.61113  0.21128   -0.31685 -0.46725  2.12700   

       Oscore   Ascore  ...  Ecstasy  Heroin  Ketamine  Legalh  LSD  Meth  \
0    -0.58331 -0.91699  ...        0       0         0       0    0     0   
1     1.43533  0.76096  ...        4       0         2       0    2     3   
2    -0.84732 -1.62090  ...        0       0         0       0    0     0   
3    -0.01928  0.59042  ...        0       0         2       0    0     0   
4    -0.45174 -0.30172  ...        1       0         0       1    0     0   
...       ...      ...  ...      ...     ...       ...     ...  ...   ...   
1880  1.88511  0.76096  ...        0       0         0       3    3     0   
1881  0.58331  0.76096  ...        2       0         0       3    5     4   
1882 -1.27553 -1.77200  ...        4       0         2       0    2     0   
1883  0.29338 -1.62090  ...        3       0         0       3    3     0   
1884  1.65653  1.11406  ...        3       0         0       3    3     0   

      Mushrooms  Nicotine  Semer  VSA  
0             0         2      0    0  
1             0         4      0    0  
2             1         0      0    0  
3             0         2      0    0  
4             2         2      0    0  
...         ...       ...    ...  ...  
1880          0         0      0    5  
1881          4         5      0    0  
1882          2         6      0    0  
1883          3         4      0    0  
1884          3         6      0    2  

[1885 rows x 32 columns]

### TEN atrybut będę badał

In [29]:
df['LSD']

0       0
1       2
2       0
3       0
4       0
       ..
1880    3
1881    5
1882    2
1883    3
1884    3
Name: LSD, Length: 1885, dtype: int64

In [31]:
values = df['LSD']

## Normalizacja

In [10]:
df = (df - df.mean())/(df.max()-df.min())

In [18]:
df_lsd = df[['Age','Gender','Education','LSD']]

##  Podział na dane treningowe i testowe

In [19]:
test_rows = random.sample(df_lsd.index.tolist(), int(round(len(df_lsd)*.2)))
train_rows=set(range(len(df_lsd))) - set(test_rows)

In [20]:
df_test = df_lsd.loc[test_rows]
df_train = df_lsd.loc[train_rows]

test_values = df_lsd.loc[test_rows]
train_values = df_lsd.loc[train_rows]

/tmp/ipykernel_16944/2404942428.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_train = df_lsd.loc[train_rows]
/tmp/ipykernel_16944/2404942428.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train_values = df_lsd.loc[train_rows]


### KNN implementacja manualna

In [21]:
kdtree = KDTree(df_lsd)

### Funkcja przydzielająca decyzję

In [22]:
def classify(query_point, k):
    _, idx =kdtree.query(query_point, k)
    return np.argmax(np.bincount(values.iloc[idx]))

In [23]:
train_predicted_values = []
train_actual_values = []

In [24]:
for _id, row in df_train.iterrows():
  prediction = classify(row,7)
  train_predicted_values.append(prediction)
  train_actual_values.append(train_values[_id])

KeyError: 0